<a href="https://colab.research.google.com/github/jenyav94/Component_Extracting/blob/master/hm3_pt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# this link won't work for you, if you want to generate working link, 
# visit https://askubuntu.com/questions/1188381/how-to-get-link-of-file-to-download-with-wget
# or download the data and place it to folder manually
! wget "https://downloader.disk.yandex.ru/disk/76973f058fe726659fa4ccc755e901552e300ab50b78634dab78a9799a22b1a2/6283c701/aZ8Akc9CqtcaAMvg9MkExUmNfU9U_vPugtmtwAC7LCh7Ob4GqD-JrqJl2rsy_hxC-1lho6HNKGxRJzyaqhvCBg%3D%3D?uid=0&filename=celebA_train_500.zip&disposition=attachment&hash=PKVHlR%2B3q03Ys8aOKp5s/dgbI1MoVhGrKn/oCUbHWox44FxZCGQEdyLSePCIwi7Tq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=application%2Fzip&owner_uid=316082386&fsize=170342993&hid=8e5296bb80b8febc2b500d8fd890bbcf&media_type=compressed&tknv=v2" -O celebA.zip
! unzip celebA.zip

In [ ]:
! git clone https://github.com/timesler/facenet-pytorch.git
! mv facenet-pytorch/models ./

Cloning into 'facenet-pytorch'...
remote: Enumerating objects: 1267, done.
remote: Total 1267 (delta 0), reused 0 (delta 0), pack-reused 1267
Receiving objects: 100% (1267/1267), 22.88 MiB | 43.39 MiB/s, done.
Resolving deltas: 100% (620/620), done.


In [ ]:
pip install pytorch-metric-learning

     |████████████████████████████████| 109 kB 5.0 MB/s 


In [ ]:
import os
import numpy as np
import random
import time
import tqdm

from PIL import Image

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import torch.nn.functional as F

import matplotlib.pyplot as plt

In [ ]:
def get_paths(dataset_type='train'):
    '''
    a function that returnes list of images paths for a given type of the dataset
    params:
      dataset_type: one of 'train', 'val', 'test'
    '''

    labels_dict = {
        'train': 0,
        'val': 1,
        'test': 2,
    }

    f = open('celebA_train/celebA_train_split.txt', 'r')
    lines = f.readlines()
    f.close()

    lines = [x.strip().split() for x in lines]
    lines = [x[0] for x in lines if int(x[1]) == labels_dict[dataset_type]]

    images_paths = []
    for filename in lines:
        images_paths.append(os.path.join('celebA_train/celebA_imgs/', filename))

    return np.array(images_paths)


class celebADataset(Dataset):

    def __init__(self, dataset_type, transform, train=True):
        self.images = get_paths(dataset_type=dataset_type)
        self.is_train = train

        f = open('celebA_train/celebA_anno.txt', 'r')
        labels = f.readlines()
        f.close()
        labels = [x.strip().split() for x in labels]
        labels = {x: y for x, y in labels}
        self.labels = [int(labels[x.split('/')[-1]]) for x in self.images]

        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):

        anchor_name = self.images[idx]
        anchor_label = self.labels[idx]
        anchor_image = self.transform(Image.open(anchor_name))

        if self.is_train:

            positive_list = [name for i, name in enumerate(self.images)
                             if self.labels[i] == anchor_label and
                             name != anchor_name]
            if len(positive_list) == 0:
                positive_name = anchor_name
            else:
                positive_name = random.choice(positive_list)

            negative_list = [name for i, name in enumerate(self.images) if self.labels[i] != anchor_label]
            negative_name = random.choice(negative_list)

            positive_image = self.transform(Image.open(positive_name))
            negative_image = self.transform(Image.open(negative_name))

            return anchor_image, positive_image, negative_image, anchor_label
        else:
            return anchor_image, anchor_label


In [ ]:
means = np.array((0.485, 0.456, 0.406))
stds = np.array((0.229, 0.224, 0.225))

transform = transforms.Compose([
    # you may add anything, e.g. augmentation
    transforms.Resize(160),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

train_data = celebADataset('train', transform)
val_data = celebADataset('val', transform)
test_data = celebADataset('test', transform)

train_mean_data = celebADataset('train', transform, False)
val_mean_data = celebADataset('val', transform, False)
test_mean_data = celebADataset('test', transform, False)


batch_size = 32
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)

train_mean_loader = torch.utils.data.DataLoader(train_mean_data, batch_size=batch_size, shuffle=True, num_workers=2)
val_mean_loader = torch.utils.data.DataLoader(val_mean_data, batch_size=batch_size, shuffle=False, num_workers=2)
test_mean_loader = torch.utils.data.DataLoader(test_mean_data, batch_size=batch_size, shuffle=False, num_workers=2)


In [ ]:
from models import inception_resnet_v1

model = inception_resnet_v1.InceptionResnetV1(pretrained='vggface2')
# model = inception_resnet_v1.InceptionResnetV1()

model.conv2d_1a.requires_grad_(False)
model.conv2d_2b.requires_grad_(False)
model.maxpool_3a.requires_grad_(False)
model.conv2d_3b.requires_grad_(False)
model.conv2d_4a.requires_grad_(False)
model.conv2d_4b.requires_grad_(False)

model.repeat_1.requires_grad_(False)
model.mixed_6a.requires_grad_(False)
model.repeat_2.requires_grad_(False)
# model.mixed_7a.requires_grad_(False)
# model.repeat_3.requires_grad_(False)
# model.block8.requires_grad_(False)


In [ ]:
def get_computing_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    return device


device = get_computing_device()


# triplet_loss = nn.TripletMarginWithDistanceLoss(distance_function=nn.PairwiseDistance())
triplet_loss = nn.TripletMarginWithDistanceLoss(distance_function=lambda x, y: 1.0 - F.cosine_similarity(x, y))
def compute_loss(anchor, positive, negative):
    return triplet_loss(anchor, positive, negative)
    # return F.cross_entropy(predictions, gt).mean()


cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)


from sklearn.neighbors import KNeighborsClassifier
def eval_model(model, data_t_generator, data_generator):
    eps = 1e-5
    accuracy = []
    model.train(False)  # disable dropout / use averages for batch_norm
    with torch.no_grad():
        X = np.zeros((8544, 512))
        y = np.zeros(8544)
        for i, (anchor_img, anchor_label) in enumerate(data_t_generator):
            anchor_img = anchor_img.to(device)
            anchor_out = model(anchor_img)

            l = i*len(anchor_img)
            r = (i+1)*len(anchor_img)
            X[l:r,:] = anchor_out.cpu().numpy()
            y[l:r] = anchor_label.cpu().numpy()

        neigh = KNeighborsClassifier(n_neighbors=5)
        neigh.fit(X, y)

        for i, (anchor_img, anchor_label) in enumerate(data_generator):
            anchor_img = anchor_img.to(device)
            anchor_out = model(anchor_img)

            y_pred = neigh.predict(anchor_out.cpu().numpy())
            accuracy.append(np.mean((anchor_label.cpu().numpy() == y_pred)))

    return np.mean(accuracy)


def train_model(model, optimizer, train_data_generator):
    train_loss = []
    model.train(True)  # enable dropout / batch_norm training behavior
    for (anchor_img, anchor_label) in tqdm.tqdm(train_data_generator):
        optimizer.zero_grad()

        anchor_img = anchor_img.to(device)
        # positive_img = positive_img.to(device)
        # negative_img = negative_img.to(device)

        anchor_out = model(anchor_img)
        # positive_out = model(positive_img)
        # negative_out = model(negative_img)

        # loss = compute_loss(anchor_out, positive_out, negative_out)

        hard_pairs = miner(anchor_out, anchor_label)
        loss = loss_func(anchor_out, anchor_label, hard_pairs)

        # backward
        loss.backward()
        optimizer.step()

        # metrics
        train_loss.append(loss.cpu().data.numpy())
    return np.mean(train_loss)


def train_loop(model, optimizer, scheduler, train_data_generator, train_eval_data_generator, val_data_generator, num_epochs):
    """
    num_epochs - total amount of full passes over training data
    """
    best_accuracy = 0
    for epoch in range(num_epochs):
        start_time = time.time()

        train_loss = train_model(model, optimizer, train_data_generator)

        val_accuracy = eval_model(model, train_eval_data_generator, val_data_generator)

        scheduler.step(val_accuracy)

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            # torch.save(model.state_dict(), 'best_model_weights2.pth')
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': val_accuracy
            }, 'best_model_weights_triplet.pth')

        # Then we print the results for this epoch:
        print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss (in-iteration): \t{:.6f}".format(train_loss))
        print("  validation accuracy: \t\t\t{:.2f} %".format(val_accuracy * 100))


In [ ]:
print(f"Our main computing device is '{device}'")

torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

model = model.to(device)

opt = torch.optim.Adam(model.parameters())

# checkpoint = torch.load('best_model_weights_triplet.pth')
# model.load_state_dict(checkpoint['model_state_dict'])
# opt.load_state_dict(checkpoint['optimizer_state_dict'])


from pytorch_metric_learning.distances import CosineSimilarity
from pytorch_metric_learning.reducers import ThresholdReducer
from pytorch_metric_learning.regularizers import LpRegularizer
from pytorch_metric_learning import miners, losses
miner = miners.MultiSimilarityMiner()
loss_func = losses.TripletMarginLoss(distance=CosineSimilarity(),
                                         reducer=ThresholdReducer(high=0.3),
                                         embedding_regularizer=LpRegularizer())


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, 'max', patience=2)


train_loop(model, opt, scheduler, train_mean_loader, train_mean_loader, val_mean_loader, num_epochs=30)

print(eval_model(model, train_mean_loader, test_mean_loader))

Our main computing device is 'cuda:0'


  9%|▉         | 25/267 [00:04<00:38,  6.35it/s]